# Leakage esitimation

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import json
from converter import convert_func
import scipy.stats as stats
from glob import glob
import os
from os.path import join

"""
1. Generate _cs_nt_Pixel_eDep.csv using Geant4
2. Save noise data as npy in data_xu_R using parameter_estimation.ipynb
3. Generate noisy movie using gamma_noise_image_convert.ipynb 
4. Estimate leakage points using leakage_estimation.ipynb
"""

%matplotlib inline

In [ ]:
gt_leaks =[
    np.array([-2.0, 1.5]),
    np.array([-0.5, 1.0]),
    np.array([1.5, 0.5])
]

## Waterdrop stereo

In [ ]:
# noisy_frames = {}
# for side in ["L", "R"]:
#     subtractor = cv2.createBackgroundSubtractorMOG2(detectShadows=False)
#     subtractor.setNMixtures(1)
    
#     fname = f"./turbine_testv5_{side}_0010-0082_noisy.mp4"
#     cap = cv2.VideoCapture(fname)
#     noisy_frames[side] = []
#     noisy_frames[side+"_fg"] = []
#     #
#     while True:
#         ret, img = cap.read()
#         if not ret:
#             break
#         fg = subtractor.apply(img)
#         noisy_frames[side].append(img)
#         noisy_frames[side+"_fg"].append(fg)

In [ ]:
# imgL, imgR = noisy_frames["L"], noisy_frames["R"]

In [ ]:
# for it_l, it_r in zip(imgL, imgR):
#     viz = np.zeros_like(it_l)
#     cv2.imshow('img_L', it_l)
#     cv2.imshow('img_R', it_r)
#     it_l = cv2.cvtColor(it_l, cv2.COLOR_BGR2GRAY)
#     it_r = cv2.cvtColor(it_r, cv2.COLOR_BGR2GRAY)
#     viz[..., 1] = it_l
#     viz[..., 2] = it_r
#     cv2.imshow('viz', viz)
#     key = cv2.waitKey(0)
#     if key == 27:
#         break
# cv2.destroyAllWindows()

In [ ]:
# fgL, fgR = noisy_frames["L_fg"], noisy_frames["R_fg"]
# for it_l, it_r in zip(fgL, fgR):
#     cv2.imshow('img_L', it_l)
#     cv2.imshow('img_R', it_r)
#     key = cv2.waitKey(0)
#     if key == 27:
#         break
# cv2.destroyAllWindows()

In [ ]:
# plt.plot(np.array(fgL).mean(0).mean(axis=0))

In [ ]:
# plt.plot(np.array(fgR).mean(0).mean(axis=0))

# Automatic raindrop detection

## Detect raindrop lines

In [ ]:
noisy_frames ={}
num_frame = 24
for side in ["L", "R"]:
    # Video to images
    noisy_frames[side] = []
    noisy_frames[side+"_color"] = []
    fname = f"./turbine_testv5_{side}_0010-0082_noisy.mp4"
    cap = cv2.VideoCapture(fname)
    while True:
        ret, img = cap.read()
        if not ret:
            break
        noisy_frames[side+"_color"].append(img)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        noisy_frames[side].append(img)

    # Obtain background image
    imgs = noisy_frames[side][:num_frame]
    bkg = np.median(imgs, axis=0).astype(np.uint8)
    noisy_frames[side+"_bkg"] = bkg
    
    # Get foreground
    noisy_frames[side+"_fg"] = []
    for it in imgs:
        diff = np.abs(it/255-bkg/255)
        noisy_frames[side+"_fg"].append(diff)

In [ ]:
# Save noisy color images for paper
for side in ["L", "R"]:
    for i in range(5, 10):
        cv2.imwrite(f"paper/exp_noisy{side}_color{i}.jpg", noisy_frames[side+"_color"][i])
        fg_img = 15*(255*noisy_frames[side+"_fg"][i]).astype(np.uint8)
        cv2.imwrite(f"paper/exp_noisy{side}_fg{i}_contrastAdj.jpg", fg_img)

In [ ]:
diff.shape

In [ ]:
import seaborn as sns
sns.set(font="Times New Roman")
sns.set_context("paper", font_scale=1.4)
# plt.style.use(['science','ieee']) #latex is required

In [ ]:
raindrop_idx = {}
fig, ax = plt.subplots(1, 2, figsize=(7, 2)) # 
for i, side in enumerate(["L", "R"]):
    plt.figure()
    col_aggre = np.array(noisy_frames[side+"_fg"]).mean(0).mean(0)
    ax[i].plot(col_aggre)
    ax[i].set_ylim([0, 0.011])
    ax[i].set_xlim([0, noisy_frames['L'][0].shape[1]])
    if i==0:
        ax[i].set_ylabel('Aggregated value')
        ax[i].set_xlabel('Column index of left images')
    else:
        ax[i].set_xlabel('Column index of right images')
    
    
    # threshold 2 sigma
    thresh = col_aggre.mean() + 2*col_aggre.std()
    print(thresh)
    ax[i].plot(np.full_like(col_aggre, thresh))
    
    # non-maximum suppression
    search_width =5 # 
    nms_idx = []
    for idx in np.where(col_aggre>thresh)[0]:
        if col_aggre[idx] == col_aggre[idx-search_width:idx+search_width+1].max():
            nms_idx.append(idx)
            
    print(nms_idx)
    raindrop_idx[side] = nms_idx

# fig.tight_layout()
# Save file
# fig.savefig("exp_column_feature.pdf", bbox_inches='tight')
fig.savefig("exp_column_feature.svg", format="svg", bbox_inches='tight')

## Find line match

In [ ]:
side_len = [(len(val), key) for key, val in raindrop_idx.items()]
features = {}
side_ascend = []
for _, side in sorted(side_len):
    print(side)
    print(raindrop_idx[side])
    features[side] = np.array(noisy_frames[side+"_fg"])[:,:,raindrop_idx[side]]
    side_ascend.append(side)

In [ ]:
# Process one by one and select match by choosing mode
match_accum = []
for t in range(features[side].shape[0]):
    print("processing:", t)
    dist_mat = features[side_ascend[0]][t].T[:, None, :] -  features[side_ascend[1]][t].T[None, :, :]
    dist_mat = np.linalg.norm(dist_mat, ord=1, axis=-1)
    match_idx = np.argmin(dist_mat, axis=1)
    match_accum.append(match_idx)
    
match_accum = np.array(match_accum)
match_ret, _ = stats.mode(match_accum, axis=0)
match_ret = match_ret.flatten().tolist()
print("Find match:", side_ascend, list(enumerate(match_ret)))

## Intersection line

In [ ]:
def parse_lines(lines):
    mat = []
    for line in lines:
        parsed_line = line[line.find('(')+1:line.find(')')].split(", ")
        data = list(map(float, parsed_line))
        mat.append(data)
    return np.array(mat)

In [ ]:
# Load blender camera info
blender_mat = {}
for fname in glob("./blender_camera/*.matrix"):
    print(fname)
    with open(fname) as f:
        lines = f.readlines()
    mat = parse_lines(lines)
    
    mat_name = os.path.split(fname)[-1].split('.')[0]
    blender_mat[mat_name] = mat

In [ ]:
blender_mat["P_L"].shape

In [ ]:
a_side = side_ascend[0]
b_side = side_ascend[1]
a_rain_idx = raindrop_idx[side_ascend[0]]
b_rain_idx = raindrop_idx[side_ascend[1]]

pred_leaks= []
for a, b in enumerate(match_ret):
    print(a_rain_idx[a], b_rain_idx[b])
    print(side_ascend)
    
    # Waterdrop estimation in image coord (x, y, 1)
    line_in_a = np.cross(np.array([a_rain_idx[a], 100, 1]), np.array([a_rain_idx[a], 0, 1]))
    print(f"line_in_a:{line_in_a}")
    line_in_b = np.cross(np.array([b_rain_idx[b], 100, 1]), np.array([b_rain_idx[b], 0, 1]))
    print(f"line_in_b:{line_in_b}")

    # Get projection matrix
    P_a = blender_mat[f"P_{a_side}"]
    P_b = blender_mat[f"P_{b_side}"]
    # Back projection PI(4x1)=P(3x4)_t * I(3x1)
    PI_a = P_a.T.dot(line_in_a)
    PI_b = P_b.T.dot(line_in_b)
    intersection_line = np.vstack([PI_a, PI_b])

    # Estimate raindrop location based on camera location
    cam_z = 3.0
    # Solve Ax=b -> intersection_line[:,:2].dot(X) = -intersection_line[:,2:].dot(np.array([cam_z, 1]))
    pred_leak = np.linalg.solve(intersection_line[:,:2], -intersection_line[:,2:].dot(np.array([cam_z, 1])))
    print("Estimation leakage:", pred_leak)
    pred_leaks.append(pred_leak)

In [ ]:
gt_leaks

## Estimation error

In [ ]:
for pred, gt in zip(pred_leaks, gt_leaks):
    print(f"L2 norm: {np.linalg.norm(pred-gt):.4f}")